In [10]:
# load the module
%load_ext autoreload
%autoreload 2

import sys 
sys.path.append("../Qtensor")
sys.path.append("../Qtensor/qtree_git")
sys.path.append("../Qtensor/qtree_git/qtree")
sys.path.append("..")


from qtensor import ZZQtreeQAOAComposer, ZZQtreeQAOAComposer_MIS, ZZQtreeQAOAComposer_MAXCUT
from qtensor import QAOAQtreeSimulator, QAOAQtreeSimulator_MIS, QAOAQtreeSimulator_MAXCUT
from qtensor.contraction_backends import TorchBackend
import Generating_Problems as Generator
from Calculating_Expectation_Values import SingleLayerQAOAExpectationValues, QtensorQAOAExpectationValuesMIS,QtensorQAOAExpectationValuesMAXCUT,QtensorQAOAExpectationValuesQUBO
from QIRO import QIRO_MIS
import RQAOA
import torch
import qtensor
import networkx as nx
import numpy as np
from scipy.optimize import minimize
import tqdm
from scipy.optimize import Bounds
import pprint
from functools import partial
import random
import json
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import networkx as nx


In [4]:
graph = nx.random_regular_graph(3, 6)
print(graph.edges())
nodes = list(graph)
nodes.remove(3)
graph2 = graph.subgraph(nodes).copy()

print(graph2.edges())


[(0, 1), (0, 3), (0, 2), (1, 4), (1, 3), (2, 4), (2, 5), (4, 5), (3, 5)]
[(0, 1), (0, 2), (1, 4), (2, 4), (2, 5), (4, 5)]


In [7]:
l = list((1, 2))
l[1]

2

In [41]:
graph = nx.random_regular_graph(3, 6)

problem = Generator.MAXCUT(graph)
expectation_values_qtensor = QtensorQAOAExpectationValuesQUBO(problem, 2, initialization='random', opt=torch.optim.SGD, opt_kwargs=dict(lr=0.0001))
#expectation_values_qtensor = QtensorQAOAExpectationValuesMAXCUT(problem, p, initialization='fixed_angles_optimization', opt=torch.optim.SGD, opt_kwargs=dict(lr=0.0001))    
#RQAOA_qtensor = RQAOA(expectation_values_qtensor, 5, type_of_problem="MAXCUT")
RQAOA_qtensor = RQAOA.RQAOA(expectation_values_qtensor, 5, type_of_problem="MAXCUT")

In [42]:
expectation_values_qtensor.optimize()

100%|██████████| 50/50 [00:07<00:00,  6.52it/s]


([3, 1], -1.0, 0.5559490567441319)

In [43]:
expectation_values_qtensor.expect_val_dict

{frozenset({1, 3}): -0.5559490567441319,
 frozenset({2, 3}): -0.31120240697005913,
 frozenset({1, 4}): -0.3112024069700591,
 frozenset({2, 4}): -0.5559490567441319,
 frozenset({2, 5}): -0.3112024069700591,
 frozenset({3, 5}): -0.31120240697005913,
 frozenset({1, 6}): -0.3112024069700591,
 frozenset({4, 6}): -0.3112024069700592,
 frozenset({5, 6}): -0.5559490567441319}

In [44]:
Corkeys = [(i, j) for i, j in expectation_values_qtensor.expect_val_dict.keys()]
Corkeys

[(1, 3), (2, 3), (1, 4), (2, 4), (2, 5), (3, 5), (1, 6), (4, 6), (5, 6)]

In [48]:
expectation_values_qtensor.expect_val_dict[frozenset(Corkeys[0])]

-0.5559490567441319

In [49]:
Corkeys.sort(key=lambda x: np.abs(expectation_values_qtensor.expect_val_dict[frozenset(x)]))
Corkeys

[(1, 4), (2, 5), (1, 6), (2, 3), (3, 5), (4, 6), (1, 3), (2, 4), (5, 6)]

In [31]:
print(RQAOA_qtensor.problem.position_translater)

[9, 1, 2, 3, 4, 5, 6]


In [30]:
RQAOA_qtensor.problem.position_translater[0] = 9

In [28]:
RQAOA_qtensor.expectation_values.problem.position_translater[0] = 8

In [32]:
print(RQAOA_qtensor.expectation_values.problem.position_translater)

[9, 1, 2, 3, 4, 5, 6]


In [17]:
print(RQAOA_qtensor.expectation_values.problem.matrix)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]]


In [18]:
RQAOA_qtensor.expectation_values.problem.matrix[0, 0] = 0

In [20]:
print(RQAOA_qtensor.problem.matrix)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]]
